In [1]:
import os
os.chdir(os.path.join(os.getcwd(),'..'))

In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt
from heatmapcluster import heatmapcluster

In [ ]:
from services import DataReader

X, y = DataReader().read_analyze_solution('bc_averaging_disease_analysis#k=0')

In [ ]:
from preprocessing import DynamicPreprocessing
import pandas as pd

pre = DynamicPreprocessing(['flux-diff', 'feature-selection','pathway-scoring', 'transport-elimination'])
df = pd.DataFrame().from_records(pre.fit_transform(X, y)).T

In [ ]:
%matplotlib inline

import seaborn as sns
sns.set(color_codes=True)

g = sns.clustermap(df)

In [ ]:
import matplotlib as plt

plt.plot()